PRAC01 Tipologia UOC Curso 2022

Scrapper de Información de Masteres

    WEB: http://www.emagister.com

Equipo: 

    Angel A. Urbina 
    Almudena Caballero

Fecha:

    07 Abril 2022

Versión: 1.4

Notas:
    
    Incluye análisis Secuencial y Multiproceso


In [1]:
# Construcción de Loggings para informar

# Objetivo: Disponer tanto en pantalla como en file (log) la información del procesamiento que se realiza 

import logging
import logging.config

# Referencia:
# https://coderzcolumn.com/tutorials/python/logging-config-simple-guide-to-configure-loggers-from-dictionary-and-config-files-in-python

# Carga de Archivo de configuracón del login
logging.config.fileConfig('loggingUOCPRA01.conf')

# Creación del logger
logger = logging.getLogger('UOCPRA01')

# Tipos de Loggers que se podrian generar:
# 'application' code
#logger.debug('debug message')
#logger.info('info message')
#logger.warning('warn message')
#logger.error('error message')
#logger.critical('critical message')

In [2]:
# Librerias Necesarias

from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import re

import os

import requests
import random
import time
from time import sleep

# Normalización textos (quitar codigos con acentos)
from unicodedata import normalize


2022-04-07 17:35:17,471 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


# 00.- Funciones Auxiliares

In [3]:
# Extraccion informacion del 'nombre archivo' de un URL que lo contiene
# Función Auxiliar

# Basado en:
# https://stackoverflow.com/questions/66876071/extracting-a-complex-substring-using-regex-with-data-from-a-string-in-python
# https://stackoverflow.com/questions/14473180/regex-to-get-a-filename-from-a-url

def ExtraeName(cadena):
    """
    Función extraccion informacion del nombre archivo de un URL
    ExtraeName(cadena)
    entrada:
        cadena --> Url
    return:
        string final del url
    """
    
    regex=r"(?<=\/)[^\/\?#]+(?=[^\/]*$)"
    matches = re.findall(regex, cadena)
    return str(matches)


In [4]:
# Generación Directorio y Devuelve Path + Archivo en el Directorio construido
# Funcion Auxiliar 

def GenDirecFile(path, filename):
    """
    GenDirecFile(path, filename, r)
    Entrada:
        path -> Path del directorio a generar
        filename -> Nombre Archivo a guardar
    Devuelve:
        path y filename
    """
    # Generación directorio si no existe
    if not os.path.exists(path):
        os.makedirs(path)
    # Grabación en el directorio en formato binario (imagenes)
    return os.path.join(path, filename)

In [5]:
# Descarga Imagenes
# Funcion Auxiliar 

def DescargaImagenes(path, link, institucion):
    """
    DescargaImagenes(path, link, institucion)
    Entrada:
        path -> Path del directorio a generar
        link -> Link a el archivo a descargar
        institucion -> Nombre de la institucion cuyo logo es la imagen
    Devuelve:
        Archivo de Imagenes descargado en el directorio path
    """
    # Delays para evitar problemas
    time.sleep(random.randint(1,2))
    # Obtener Imagen
    #r = requests.get(s, allow_redirects=False)   
    r = requests.get(link, allow_redirects=True)
    # Generar Nombre Archivo
    nombre = str(institucion)
    # Directorio donde se generara salida
    file01=GenDirecFile(str(path), nombre)
    # Grabacion resultados
    with open(file01, 'wb') as file:
        file.write(r.content)

In [6]:
# Grabación Archivo en Directorio
# Funcion Auxiliar 

# path = IMAGENES
# filename = img_alt + '.jpg'
# buffer = r.content
# open(nombre, 'wb').write(r.content)

def GraFileDirec(path, filename, r):
    """
    GraFileDirec(path, filename, r)
    path -> Path del directorio a generar
    filename -> Nombre Archivo a guardar
    r --> Objeto BeatifulSoup
        r.content -> Contenido Stream del Objeto
    """
    # Generación directorio si no existe
    if not os.path.exists(path):
        os.makedirs(path)
    # Grabación en el directorio en formato binario (imagenes)
    with open(os.path.join(path, filename), 'wb') as temp_file:
        temp_file.write(r.content)

# 01.- Análisis de Links asociados a la WEB

In [7]:
# 01.01- El primer paso es la identificación de los links asociados a cada Master

# Pagina WEB a analizar

URLWebPageAnalizar = 'https://www.emagister.com/master/web/sitemap'

logger.info('Analisis WEB %s', URLWebPageAnalizar)

# Lista donde se almacenaran los links encontrados
ListaWebs=[]

# Abrir Web
html = urlopen(URLWebPageAnalizar)

# Generar objeto BeatifulSoup
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find_all('a'):
    # Busca todos los elementos que esten en 'href'
    try:
        StringWeb = link.attrs['href']
    except:
        continue
    # Comprueba que lo obtenido se corresponda con el formato de una pagina WEB
    # match (True si es un link / False en resto)
    match = re.search(r'https?://[^\s<>"]+|www\.[^\s<>"]+', str(StringWeb)) 
    if match:      
        ListaWebs.append(link.attrs['href'])
        logger.info('Encontrado link %s', link.attrs['href'])

# Guardar links obtenidos en Archivo Auxiliar FILE_Links
# - Nombre Archivo
FILE_Links = '01_ListaWebs.xlsx'
# - Genera DataFrame apartir de la lista de WEBs generadas
df = pd.DataFrame(ListaWebs)
# - Escribe Archivo
df.to_excel(FILE_Links)
logger.info('Generado Archivo %s', FILE_Links)

2022-04-07 17:39:04,921 - UOCPRA01 - INFO - Analisis WEB https://www.emagister.com/master/web/sitemap
2022-04-07 17:39:05,452 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/web/pagina/legal#sixth
2022-04-07 17:39:05,453 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/
2022-04-07 17:39:05,453 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/
2022-04-07 17:39:05,453 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/
2022-04-07 17:39:05,454 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/universidades/
2022-04-07 17:39:05,455 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/
2022-04-07 17:39:05,455 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/formacion-profesional/
2022-04-07 17:39:05,456 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/oposiciones-convocatorias/
2022-04-07 17:39:05,457 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/subvenciona

2022-04-07 17:39:05,491 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-pedagogia-tematica-479.htm
2022-04-07 17:39:05,492 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-preparacion-examenes-tematica-826.htm
2022-04-07 17:39:05,493 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-arte-diseno-estetica-tematica-54.htm
2022-04-07 17:39:05,493 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-artes-plasticas-tematica-421.htm
2022-04-07 17:39:05,493 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-artesania-tematica-512.htm
2022-04-07 17:39:05,494 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-creacion-audiovisual-tematica-419.htm
2022-04-07 17:39:05,494 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-diseno-tematica-72.htm
2022-04-07 17:39:05,494 - UOCPRA01 - INFO - Encontrado link https://www.emagi

2022-04-07 17:39:05,529 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-investigacion-cientifica-tematica-476.htm
2022-04-07 17:39:05,529 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-matematicas-tematica-472.htm
2022-04-07 17:39:05,530 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-quimica-tematica-32.htm
2022-04-07 17:39:05,530 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-inmobiliaria-construccion-tematica-528.htm
2022-04-07 17:39:05,531 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-arquitectura-tematica-526.htm
2022-04-07 17:39:05,531 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-construccion-tematica-758.htm
2022-04-07 17:39:05,532 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/master/master-gestion-inmobiliaria-tematica-179.htm
2022-04-07 17:39:05,532 - UOCPRA01 - INFO - Encontrado link http

2022-04-07 17:39:05,571 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com/centros.htm
2022-04-07 17:39:05,572 - UOCPRA01 - INFO - Encontrado link https://formacio.lavanguardia.com/
2022-04-07 17:39:05,573 - UOCPRA01 - INFO - Encontrado link https://www.emagister.fr/
2022-04-07 17:39:05,573 - UOCPRA01 - INFO - Encontrado link https://www.emagister.co.uk/
2022-04-07 17:39:05,574 - UOCPRA01 - INFO - Encontrado link https://www.emagister.it/
2022-04-07 17:39:05,575 - UOCPRA01 - INFO - Encontrado link https://www.emagister.de/
2022-04-07 17:39:05,575 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com.mx/
2022-04-07 17:39:05,577 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com.co/
2022-04-07 17:39:05,578 - UOCPRA01 - INFO - Encontrado link https://www.emagister.cl/
2022-04-07 17:39:05,579 - UOCPRA01 - INFO - Encontrado link https://www.emagister.com.ar/
2022-04-07 17:39:05,579 - UOCPRA01 - INFO - Encontrado link https://www.emagister.in/
2022-04-07 17:39:0

In [8]:
# Nos informa del Número Total de Links encontrados
logger.info('Encontrado %s links', len(ListaWebs))

2022-04-07 17:39:13,205 - UOCPRA01 - INFO - Encontrado 206 links


In [9]:
# 01.02.- Función para Construcción del Diccionario de Información de los links encontrados.

diccionario={}

def RecorrerWebs(ListaWebs):
    """
    Construye Diccionario Valores de una lista de Webs
    
    Funcion:
       RecorrerWebs( ListaWeb)
    Entrada:
        ListaWeb --> Lista de Direcciones de Web
    Salida:
        Diccionario con información:
            Keys: 
                Indice correlativo
            Values:
                - Pagina donde buscamos
                - Titulo Master
                - Link al master concreto
    """
    
    # Leer Webs obtenidas paso previo
    # - indice
    indice = 0
    # - Delay 1 seg para no ser banneado
    sleep(1)
    
    for indi, ListaWeb in enumerate(ListaWebs):
        # Delay 1 seg para no ser banneado
        sleep(1)
        # Captura de posibles errores Webs
        try:
            html = urlopen(ListaWeb)
        except HTTPError as e:
            # Lanza un aviso con indice de ListaWebs y error
            logger.warning('%s warning %s',indi, e)
            continue
        except URLError as e:
            # Lanza aviso si no encuentra la Web
            logger.warning('%s ',ListaWeb)
            logger.warning('%s ',e)               
            continue
        # Captura posibles errores Beatifulsoup
        try:
            bs = BeautifulSoup(html, 'html.parser')
        except AttributeError as e:
            logger.warning('%s warning %s',indi, e)              
            continue
        
        # Captura de información básica
        divs = bs.find_all(class_ ="course-box-item title-box")
    
        for s in divs:
            inks = s.find_all('h3')
            for t in inks:
                lista = t.find_all('a')
                # Obtención indice y valor de lista
                #for indice, x in enumerate(lista):
                for x in lista:
                    indice = indice+1
                    # Titulo 
                    x2 = x.attrs['title']
                    # Referencia al titulo
                    x1 = x.attrs['href']
                    logger.info('%s: %s',indice, x2)

                    # Construccion diccionario salida
                    diccionario[indice] = [ListaWeb,x2,x1]
    return diccionario

In [10]:
# 01.03.- Construcción Efectiva del Diccionario

# Nombre Diccionario Construido:
#    a --> diccionario

a = RecorrerWebs(ListaWebs)

logger.info('Construido Diccionario')

2022-04-07 17:39:51,588 - UOCPRA01 - INFO - 1: Máster MBA en dirección de centros hidrotermales
2022-04-07 17:39:51,588 - UOCPRA01 - INFO - 2: Máster en Nutrición y Salud
2022-04-07 17:39:51,589 - UOCPRA01 - INFO - 3: Máster en Biotecnología de la Salud - Madrid
2022-04-07 17:39:51,589 - UOCPRA01 - INFO - 4: Máster Universitario en Nutrición y Salud
2022-04-07 17:39:51,590 - UOCPRA01 - INFO - 5: Máster Profesional en Gestión Sanitaria
2022-04-07 17:39:51,590 - UOCPRA01 - INFO - 6: Técnico en Salud Mental (10 créditos ECTS)
2022-04-07 17:39:51,591 - UOCPRA01 - INFO - 7: MÁSTER EN SALUD MENTAL EN EL ANCIANO + MÁSTER EN LA ENFERMEDAD DEL ALZHEIMER
2022-04-07 17:39:51,592 - UOCPRA01 - INFO - 8: MÁSTER EN SALUD MENTAL
2022-04-07 17:39:51,593 - UOCPRA01 - INFO - 9: Máster en Nutrición y Salud - A distancia
2022-04-07 17:39:51,594 - UOCPRA01 - INFO - 10: Máster en psicología clínica y de la salud
2022-04-07 17:39:51,594 - UOCPRA01 - INFO - 11: Máster en trabajo social en servicios de la salud

2022-04-07 17:39:57,499 - UOCPRA01 - INFO - 72: MÁSTER EN PATOLOGÍAS TRAUMATOLÓGICAS Y TRATAMIENTO PARA FISIOTERAPEUTAS - CON CERTIFICACIÓN UNIVERSITARIA INTERNACIONAL Y RECONOCIMIENTO DE 30 ECTS
2022-04-07 17:39:57,500 - UOCPRA01 - INFO - 73: Máster de formación permanente en fisioterapia deportiva
2022-04-07 17:39:57,500 - UOCPRA01 - INFO - 74: DOBLE MÁSTER EN FISIOTERAPIA Y REHABILITACIÓN DEPORTIVA - ONLINE
2022-04-07 17:39:57,501 - UOCPRA01 - INFO - 75: Master en Fisioterapia Deportiva
2022-04-07 17:39:57,501 - UOCPRA01 - INFO - 76: MASTER EN FISIOTERAPIA + REHABILITACIÓN DEPORTIVA - Online
2022-04-07 17:39:57,502 - UOCPRA01 - INFO - 77: Doctorado Doctrinal en Desarrollo de Excelencia en Kinesiologia y Fisioterapia
2022-04-07 17:39:57,502 - UOCPRA01 - INFO - 78: MÁSTER EN DIRECCIÓN DE CLÍNICAS DE FISIOTERAPIA
2022-04-07 17:39:57,503 - UOCPRA01 - INFO - 79: Postgrado en actividades de fisioterapia y rehabilitación en los mayores
2022-04-07 17:39:57,503 - UOCPRA01 - INFO - 80: Máster

2022-04-07 17:40:05,427 - UOCPRA01 - INFO - 151: Máster en homeopatía, fitoterapia y nutrición
2022-04-07 17:40:05,428 - UOCPRA01 - INFO - 152: Postgrado de monitor de yoga terapéutico
2022-04-07 17:40:05,428 - UOCPRA01 - INFO - 153: MÁSTER EN HOMEOPATÍA, REHABILITACIÓN Y NUTRICIÓN + MÁSTER EN DIETÉTICA Y COACH NUTRICIONAL
2022-04-07 17:40:05,429 - UOCPRA01 - INFO - 154: Máster en Reflexología Corporal
2022-04-07 17:40:05,429 - UOCPRA01 - INFO - 155: Máster en homeopatía, fitoterapia y nutrición
2022-04-07 17:40:05,430 - UOCPRA01 - INFO - 156: MÁSTER EN FITOTERAPIA - CERTIFICADO POR APENB - ASOCIACIÓN PROFESIONAL ESPAÑOLA DE NATUROPATÍA Y BIOTERAPIA -
2022-04-07 17:40:05,431 - UOCPRA01 - INFO - 157: MÁSTER EN FARMACOLOGÍA CLÍNICA PARA FARMACIA - CON CERTIFICACIÓN UNIVERSITARIA INTERNACIONAL Y RECONOCIMIENTO DE 30 ECTS
2022-04-07 17:40:05,431 - UOCPRA01 - INFO - 158: Máster de experto en radiestesia, geobiología y feng shui
2022-04-07 17:40:05,432 - UOCPRA01 - INFO - 159: Máster en herb

2022-04-07 17:40:11,330 - UOCPRA01 - INFO - 227: Máster en Servicios Técnicos Funerarios
2022-04-07 17:40:13,082 - UOCPRA01 - INFO - 228: Posturopodia - Podoposturología I. Regulación Postural y Captor Podal
2022-04-07 17:40:13,083 - UOCPRA01 - INFO - 229: Experto Universitario en Podología Basada en la Práctica Clínica
2022-04-07 17:40:13,083 - UOCPRA01 - INFO - 230: Experto en Podología Deportiva y Ortopodología
2022-04-07 17:40:13,083 - UOCPRA01 - INFO - 231: Máster Universitario en Investigación en Podología (ONLINE)
2022-04-07 17:40:13,084 - UOCPRA01 - INFO - 232: Máster Universitario en Podología Deportiva
2022-04-07 17:40:13,085 - UOCPRA01 - INFO - 233: Técnico en Pedicure Clínico
2022-04-07 17:40:13,085 - UOCPRA01 - INFO - 234: Experto universitario en ortopedia
2022-04-07 17:40:13,086 - UOCPRA01 - INFO - 235: Experto Universitario en Podología Deportiva y Ortopodología
2022-04-07 17:40:13,087 - UOCPRA01 - INFO - 236: Experto en Podología Basada en la Práctica Clínica
2022-04-0

2022-04-07 17:40:20,589 - UOCPRA01 - INFO - 302: Master Dirección de Centros y Gestión de Servicios Sociosanitarios para Mayores
2022-04-07 17:40:20,590 - UOCPRA01 - INFO - 303: Máster en Hipología: el Caballo Como Medio de Revitalización y Rehabilitación
2022-04-07 17:40:20,590 - UOCPRA01 - INFO - 304: EXPERTO EN PROGRAMAS DE ATENCIÓN Y PREVENCIÓN EN LA TERCERA EDAD TERAPIA OCUPACIONAL
2022-04-07 17:40:20,591 - UOCPRA01 - INFO - 305: Máster en Terapia Ocupacional Aplicada
2022-04-07 17:40:20,591 - UOCPRA01 - INFO - 306: MÁSTER EN TERAPIA DE PAREJA + MÁSTER EN COUNSELING Y TERAPIA GESTALT - CON PRÁCTICAS GARANTIZADAS
2022-04-07 17:40:22,548 - UOCPRA01 - INFO - 307: Máster en análisis clínicos en laboratorio para veterinaria
2022-04-07 17:40:22,549 - UOCPRA01 - INFO - 308: MÁSTER EN URGENCIAS VETERINARIAS Y QUIRÓFANO
2022-04-07 17:40:22,549 - UOCPRA01 - INFO - 309: Máster en urgencias veterinarias y quirófano
2022-04-07 17:40:22,550 - UOCPRA01 - INFO - 310: TÉCNICO SUPERIOR VETERINARIO 

...

2022-04-07 17:47:11,226 - UOCPRA01 - INFO - 2696: Master en Gestión de la Inmigración
2022-04-07 17:47:11,227 - UOCPRA01 - INFO - 2697: Máster Oficial - Integración Europea
2022-04-07 17:47:11,228 - UOCPRA01 - INFO - 2698: Doctor of Philosophy in Public Management, Ph.D. / Doctorado en Gestión Pública, PHD
2022-04-07 17:47:13,466 - UOCPRA01 - INFO - 2699: Curso Experto de Posgrado en Innovación Pública
2022-04-07 17:47:13,467 - UOCPRA01 - INFO - 2700: Máster universitario en dirección en la gestión pública
2022-04-07 17:47:13,468 - UOCPRA01 - INFO - 2701: Maestría en Estudios Interdisciplinarios sobre Desarrollo
2022-04-07 17:47:16,164 - UOCPRA01 - INFO - 2702: Máster oficial en ciencia política y gestión pública (URJC)
2022-04-07 17:47:16,165 - UOCPRA01 - INFO - 2703: MÁSTER EN ENSAYOS FISICOQUÍMICOS
2022-04-07 17:47:16,165 - UOCPRA01 - INFO - 2704: Diplomado en Administración Pública
2022-04-07 17:47:16,166 - UOCPRA01 - INFO - 2705: Experto Universitario en Gestión de Documentación A

2022-04-07 17:47:27,069 - UOCPRA01 - INFO - 2772: Máster en Gestión y Dirección Logística
2022-04-07 17:47:27,070 - UOCPRA01 - INFO - 2773: Posgrado en Logística y Supply Chain con Doble Titulación
2022-04-07 17:47:27,070 - UOCPRA01 - INFO - 2774: Master Universitario en Logística
2022-04-07 17:47:27,071 - UOCPRA01 - INFO - 2775: Máster en Gestión y Dirección Logística-
2022-04-07 17:47:27,072 - UOCPRA01 - INFO - 2776: Master Logística y Dirección de Almacén + Titulación Universitaria
2022-04-07 17:47:27,073 - UOCPRA01 - INFO - 2777: Máster en logística más experto SAP en logística y materiales
2022-04-07 17:47:27,073 - UOCPRA01 - INFO - 2778: Máster de formación permanente en logística y transporte
2022-04-07 17:47:27,074 - UOCPRA01 - INFO - 2779: Master MBA en Dirección de Operaciones, Logística y Producción + Titulación Universitaria
2022-04-07 17:47:27,075 - UOCPRA01 - INFO - 2780: Experto Universitario en Gestión de la Cadena de Suministro y Logística 4.0
2022-04-07 17:47:27,075 -

2022-04-07 17:47:45,978 - UOCPRA01 - INFO - 2860: Curso de Coreano en el Extranjero para para profesionales
2022-04-07 17:47:45,978 - UOCPRA01 - INFO - 2861: Curso de Coreano en Seúl
2022-04-07 17:47:45,979 - UOCPRA01 - INFO - 2862: Coreano Inicial SÁBADOS.
2022-04-07 17:47:45,979 - UOCPRA01 - INFO - 2863: Clases particulares de coreano
2022-04-07 17:47:45,980 - UOCPRA01 - INFO - 2864: Curso de coreano inicial (Online y presencial)
2022-04-07 17:47:51,830 - UOCPRA01 - INFO - 2865: Máster Universitario en Enseñanza del Español como lengua extranjera
2022-04-07 17:47:51,831 - UOCPRA01 - INFO - 2866: Máster Universitario en Enseñanza del Español como Lengua Extranjera
2022-04-07 17:47:51,832 - UOCPRA01 - INFO - 2867: Máster Universitario en la Enseñanza del Español como Lengua Extranjera
2022-04-07 17:47:51,832 - UOCPRA01 - INFO - 2868: Máster en Didáctica del Español como Lengua Extranjera (60 ECTS)Otorgado por la Universidad San Jorge
2022-04-07 17:47:51,833 - UOCPRA01 - INFO - 2869: Má

2022-04-07 17:48:09,079 - UOCPRA01 - INFO - 2940: Curso japonés regular
2022-04-07 17:48:09,080 - UOCPRA01 - INFO - 2941: Japonés 1 ONLINE
2022-04-07 17:48:09,080 - UOCPRA01 - INFO - 2942: Japonés de Principiantes a Avanzados
2022-04-07 17:48:09,081 - UOCPRA01 - INFO - 2943: Curso básico online de japonés
2022-04-07 17:48:09,081 - UOCPRA01 - INFO - 2944: Curso Intensivo de Japonés Básico (Nivel Oficial del Consejo Europeo A1-A2)
2022-04-07 17:48:09,082 - UOCPRA01 - INFO - 2945: Japonés inicial intensivo
2022-04-07 17:48:09,083 - UOCPRA01 - INFO - 2946: Curso Intensivo Japonés Nivel Inicial, Básico, Medio, Avanzado
2022-04-07 17:48:09,083 - UOCPRA01 - INFO - 2947: Curso individual de japonés
2022-04-07 17:48:09,084 - UOCPRA01 - INFO - 2948: Curso de Japonés en Tokio
2022-04-07 17:48:09,085 - UOCPRA01 - INFO - 2949: Cursos de Japonés en Tokio
2022-04-07 17:48:09,085 - UOCPRA01 - INFO - 2950: Japonés Principiante
2022-04-07 17:48:09,086 - UOCPRA01 - INFO - 2951: Japonés inicial y elementa

2022-04-07 17:48:24,157 - UOCPRA01 - INFO - 3031: Máster Internacional Online en Paz Seguridad y Defensa
2022-04-07 17:48:24,158 - UOCPRA01 - INFO - 3032: Máster en derecho administrativo
2022-04-07 17:48:24,158 - UOCPRA01 - INFO - 3033: Máster experto en seguridad ciudadana
2022-04-07 17:48:24,159 - UOCPRA01 - INFO - 3034: Máster en investigación de incendios forestales
2022-04-07 17:48:24,159 - UOCPRA01 - INFO - 3035: MÁSTER EN SEGURIDAD EN PARQUES INFANTILES
2022-04-07 17:48:24,160 - UOCPRA01 - INFO - 3036: MÁSTER EN CONTAMINACIÓN ACÚSTICA + PÉRITO JUDICIAL EN CONTAMINACIÓN ACÚSTICA
2022-04-07 17:48:24,161 - UOCPRA01 - INFO - 3037: MÁSTER EN PERITO JUDICIAL EN CONTAMINACIÓN ACÚSTICA
2022-04-07 17:48:24,162 - UOCPRA01 - INFO - 3038: Doctorado internacional en prevención y gestión de riesgos laborales
2022-04-07 17:48:24,162 - UOCPRA01 - INFO - 3039: Doctorado internacional en estudios de terrorismo y corrupción
2022-04-07 17:48:26,460 - UOCPRA01 - INFO - 3040: Máster en intervención 

01.04.- Guardamos la información obtenida en un Archivo para ello:

        01.04.01.- Generamos Dataframe
        01.04.02.- Generamos File a partir del Dataframe

 Pasos:

In [11]:
# 01.04.01.- Generación de Dataframe a partir del Diccionario

# Referencia:
# https://stackoverflow.com/questions/54416620/dictionary-with-multiple-key-values-to-dataframe

TablaResult = (pd.DataFrame.from_dict(a, orient='index')
                  .rename(columns=lambda x: x+1)
                  .add_prefix('n')
                  .rename_axis('indice')
                  .reset_index())

logger.info('Construida Tabla TablaResult')

# Poner nombres columnas
TablaResult.columns = ['key','Pagina_Origen', 'Titulo', 'Pag_Referencia']

# Poner de indice a key
TablaResult=TablaResult.set_index('key')

# Verificación Construcción Ok( No necesario )
# TablaResult.head(3)

logger.info('Tamaño tabla  %s', TablaResult.shape)
logger.info('Columnas  %s', TablaResult.columns)

2022-04-07 17:49:21,832 - UOCPRA01 - INFO - Construida Tabla TablaResult
2022-04-07 17:49:21,838 - UOCPRA01 - INFO - Tamaño tabla  (3093, 3)
2022-04-07 17:49:21,839 - UOCPRA01 - INFO - Columnas  Index(['Pagina_Origen', 'Titulo', 'Pag_Referencia'], dtype='object')


In [12]:
# 01.04.02.- Generación Archivo parcial de resultados

# Generación Nombres Archivos a generar
from datetime import datetime

nombre=datetime.today().strftime('%d-%m-%y')

FILE_01 = "LINK-"+nombre+".xlsx"

TablaResult.to_excel(FILE_01)
logger.info('Generado Archivo %s', FILE_01)

2022-04-07 17:49:22,155 - UOCPRA01 - INFO - Generado Archivo LINK-07-04-22.xlsx


# 02.- Analisis de Links Identificados 

    02.1- Para ello leeremos Archivo del paso previo
    02.2- Haremos el analisis

In [14]:
# 02.01.- Identificación Archivos en nuestra Maquina

# Directorio donde estamos
cwd=os.getcwd()
print("-----------------------------------------------------------------------")
print("A.- Directorio en maquina donde estamos")
print (cwd)

# Lista de Archivos disponibles en directorio 'Directorio'
Directorio = cwd
print("-----------------------------------------------------------------------")
print("C.- Listado de Archivos Disponibles")
ListaFile=os.listdir(Directorio)

# Selección Archivos que hayan sido generados en lanzamientos previos del pto 01.
# - Uso Regex 
cadena=r"^LINK-"
prog1 = re.compile(cadena)

# Extraer lista de archivos que cumplan criterio

Archivos_FILT=[]
for s in ListaFile:
    if prog1.search(s):
        Archivos_FILT.append(s)
# Listado de Nombres Archivos validos
Archivos_FILT

-----------------------------------------------------------------------
A.- Directorio en maquina donde estamos
/home/anson/Descargas/01_UOC/05_TIPOLOGIA/PRA01
-----------------------------------------------------------------------
C.- Listado de Archivos Disponibles


['LINK-07-04-22.xlsx', 'LINK-05-04-22.xlsx']

In [15]:
# Escogeremos el archivo que nos interese de los identificados en el paso previo
#     Este paso permite:
#            desacoplar la busqueda de Links asociados a el analisis de los Links

# Leer Archivo
FILE_02 = 'LINK-07-04-22.xlsx'

df = pd.read_excel(FILE_02, index_col=None)

logger.info('Leido Archivo %s', FILE_02)
logger.info('Numero de registros en Archivo %s', len(df['key']))

2022-04-07 17:51:18,994 - UOCPRA01 - INFO - Leido Archivo LINK-07-04-22.xlsx
2022-04-07 17:51:18,995 - UOCPRA01 - INFO - Numero de registros en Archivo 3093


In [22]:
# Función Lectura Información Webs secundarias de Masters

def WebSecundaria(Link):
    """
    Funcion:
        WebSecundaria(link, indice)
        Version 2.0
    Entrada:
        link -> Link a analizar
     
    Return:
        Lista valores obtenidos
        - Titulo -> Titulo Master
        - Entidad -> Nombre entidad que hace master
        - urlImagen -> Link Imagen
        - nombreImagen -> Nombre Imagen
        - Telefono -> Telefono contacto
        - Precio -> Precio Master
        - Metodologia -> Metodologia Master
        - Lugar -> Lugar Imparticion
        - Duracion -> Duracion Master
        - Tipologia -> Tipologia Master
        - BolsaEmp -> Bolsa Empleo
        - x3 -> Precio Master
        - x4 -> Texto Descriptivo Master
        - x5 -> Tipologia Master
        - x6 -> Programa Master
    """
    # Abrir Web    
    # Captura de posibles errores Webs
    
    # Delays para evitar problemas
    time.sleep(random.randint(1,2))
    
    try:
        html = urlopen(Link)
    except HTTPError as e:
        # Lanza un aviso con indice de ListaWebs y error
        logger.warning('warning %s', e)
        return
    except URLError as e:
        # Lanza aviso si no encuentra la Web
        logger.warning('%s ',e)               
        return

    # Generar objeto BeatifulSoup
    bs = BeautifulSoup(html, 'html.parser')
                       
    # Titulo Master
    for t1 in bs.select(".title-box__name"):
        #print(t1.text)
        x1 = t1.text
        Titulo = x1.strip()
                       
    # Nombre entidad que hace el Master
    for t1 in bs.select(".course-box__link"):
        #print(t1.text)
        Entidad = t1.text.strip()
        
    # Imagen en pagina Master
    a=bs.select(".boxes-untrack__logo > img:nth-child(1)")
    # Si hay imagen
    if a:
        # Link Imagen
        urlImagen=a[0]['src'].strip()
        # Extraer nombre Archivo Imagen
        nombreImagen= ExtraeName(urlImagen)
    else:
        urlImagen=""
        nombreImagen="Sin Imagen"

    # Telefono
    a=bs.select(".app_contactCenterSectionDesktop:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > a:nth-child(1) > span:nth-child(1)")
    if a:
        # Extracción Telefono
        Telefono=a[0].text
        # - Elimina caracteres especiales
        Telefono=re.sub(r"[^a-zA-Z0-9]","",Telefono)
        # - Elimina espacios blanco principio y final
        Telefono=Telefono.strip()
    else:
        Telefono="Sin Datos"
        
     # Precio
    a=bs.select(".app_course_price_box > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)")
    if a:
        # Extracción Precio
        Precio=a[0].text
        # En casos de precios formato "XXXX-YYYY" nos quedamos con XXXX
        Precio=Precio.split("-")[0]
        # - Elimina caracteres especiales
        Precio=re.sub(r"[^a-zA-Z0-9]","",Precio)
        # - Elimina espacios blanco principio y final
        Precio=Precio.strip()
        # - Si todos los caracteres son numericoa convierte a float
        if Precio.isnumeric():
            # Convertir en float
            Precio=float(Precio)
        
    else:
        Precio="Precio a consultar"
    
    # Metodologia
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(2) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Metodologia
        Metodologia=a[0].text.strip()
    else:
        Metodologia="Sin Datos"
    
    # Lugar impartición Master
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(3) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Lugar
        Lugar=a[0].text.strip()
    else:
        Lugar="Sin Datos"
    
    # TipoCurso
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(4) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # TipoCurso
        TipoCurso=a[0].text.strip()
    else:
        TipoCurso="Sin Datos"
       
    # Tipologia
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(1) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Tipologia
        Tipologia=a[0].text.strip()
    else:
        Tipologia="Sin Datos" 

     # Bolsa Empleo
    a=bs.select("ul.course-detail__list:nth-child(2) > li:nth-child(4) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Bolsa Empleo (Si/No)
        BolsaEmpl=a[0].text.strip()
    else:
        BolsaEmpl="Sin Datos" 
                       
    # Precio del Master
    # - Si no tiene valor asigna precio Sin Datos
    pa = bs.select("div.price-box:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1)")
    if not pa:
        x3= "SinDatos"
    # - En el resto de casos
    for t1 in pa:
        # - Obtención del valor
        x31 = t1.text
        # En casos de precios formato "XXXX-YYYY" nos quedamos con XXXX
        x31=x31.split("-")[0]        
        # - Elimina caracteres especiales
        x3=re.sub(r"[^a-zA-Z0-9]","",x31)
        # - Elimina espacios blanco principio y final
        x3=x3.strip()
        # - Si todos los caracteres son numericoa convierte a float
        if x3.isnumeric():
            # Convertir en float
            x3=float(x3)
            # Definirlo como precio ( La información esta en dos sitios )
            Precio=x3
        
    # Texto Descriptivo
    # - Si no tiene valor asigna cadena vacia
    if not bs.select(".course-box__text"):
        DESCR="SinDatos"
    # - En el resto de casos
    for t1 in bs.select(".course-box__text"):
        # - Obtención del valor
        x41 = t1.text
        # - Eliminamos acentos. Basado en:
# https://es.stackoverflow.com/questions/135707/c%C3%B3mo-puedo-reemplazar-las-letras-con-tildes-por-las-mismas-sin-tilde-pero-no-l
        # -> NFD y eliminar diacríticos
        x4 = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize("NFD", x41), 0, re.I)
        # -> NFC
        x42 = normalize( 'NFC', x4)
        
        # - Eliminamos caracteres especiales
        x4=re.sub(r"[^a-zA-Z0-9]"," ",x42)
        # - Elimina espacios blanco principio y final
        DESCR=x4.strip()
        
        
    # Tipologia del Master
    # - Si no tiene valor asigna cadena vacia
    if not bs.select(".course-venues__address > span:nth-child(1)"):
        x5=""
    # - En el resto de los casos
    for t1 in bs.select(".course-venues__address > span:nth-child(1)"):
        # Si no hay valores
        if not t1:
            x5="Sin Datos"
        #print(t1.text)
        x5 = t1.text.strip()

    # Programa Master
    texto=[]
    for table in bs.select(".lessons-box.app_lessons-box"):
        for i, tr in enumerate(table.findAll('strong')):
            texto.append(tr.text)
            for td in tr.findAll('li'):
                texto.append(td.text)
    PROGRAMA = texto
                       
    # Construcción Lista resultado 
#        - Titulo -> Titulo Master -- 'MASTER'                   
#        - Entidad -> Nombre entidad que hace master -- 'ENTIDAD'
#        - Precio -> Precio Master -- 'PRECIO'
#        - TipoCurso -> Tipo Curso -- 'TIPO_CURSO'
#        - Tipologia -> Tipologia Master -- 'TIPOLOGIA'
#        - Metodologia -> Metodologia Master -- 'METODOLOGIA'
#        - Lugar -> Lugar Imparticion -- LUGAR
#        - BolsaEmp -> Bolsa Empleo -- 'BOLSAEMP'
#        - Telefono -> Telefono contacto --  'TELEFONO'
#        - x4 -> Texto Descriptivo Master
#        - x6 -> Programa Master  
#        - urlImagen -> Link Imagen
#        - nombreImagen -> Nombre Imagen
#        'WEB'

#        - x3 -> Precio Master (NO LO USAREMOS)
#        - x5 -> Tipologia Master (NO LO USAREMOS)
  
    
    logger.info('Procesada %s', x1)                  
    return (Titulo,
            Entidad,
            Precio,
            Tipologia,
            Metodologia,           
            Lugar,          
            BolsaEmpl,
            Telefono,
            DESCR,
            PROGRAMA,
            urlImagen,
            nombreImagen,           
            Link)

# 03.- ANALISIS

Tenemos dos opciones:
    
    03.01- SECUENCIAL
    03.02.-MULTIPROCESO

In [23]:
# 03.01.-PROCESO SECUENCIAL de generación Datos Master

# Datos --> DataFrame df
#  - Columnas:
#       'key'-> Indice
#       'Pagina_Origen' -> Web Origen Busqueda
#     , 'Titulo' -> Nombre Master
#     , 'Pag_Referencia' --> Web de detalle Master
#     

# Construcción Lista Webs a analizar
listadoWebs = df['Pag_Referencia'].tolist()
# Tamaño Datos
#tamano = len(listadoWebs)
tamano = 5

# Control Tiempo
start = time.time()

logger.info('********************************')
logger.info('*   INICIO PROCESO SECUENCIAL  *')
logger.info('********************************')

# Para cada valor del indice
for indice in range(tamano):
    # Paro para evitar bloqueo 
    sleep(1)
    # Construcción Diccionario
    #s = WebSecundaria(listadoWebs[indice], indice)
    resultado_seq = WebSecundaria(listadoWebs[indice])
    #logger.info('%s: Informacion de %s', indice, s[indice][2])

logger.info('********************************')
logger.info('*     FIN PROCESO SECUENCIAL   *')
logger.info('********************************')

end = time.time()
duracion = (end-start)
logger.info('El proceso secuencial duro %s segundos', duracion)

2022-04-07 19:54:24,692 - UOCPRA01 - INFO - ********************************
2022-04-07 19:54:24,695 - UOCPRA01 - INFO - *   INICIO PROCESO SECUENCIAL  *
2022-04-07 19:54:24,696 - UOCPRA01 - INFO - ********************************
2022-04-07 19:54:29,118 - UOCPRA01 - INFO - Procesada  Máster MBA en dirección de centros hidrotermales 
2022-04-07 19:54:33,101 - UOCPRA01 - INFO - Procesada  Máster en Nutrición y Salud 
2022-04-07 19:54:36,895 - UOCPRA01 - INFO - Procesada  Máster en Biotecnología de la Salud - Madrid 
2022-04-07 19:54:40,829 - UOCPRA01 - INFO - Procesada  Máster Universitario en Nutrición y Salud 
2022-04-07 19:54:43,631 - UOCPRA01 - INFO - Procesada  Máster Profesional en Gestión Sanitaria 
2022-04-07 19:54:43,632 - UOCPRA01 - INFO - ********************************
2022-04-07 19:54:43,632 - UOCPRA01 - INFO - *     FIN PROCESO SECUENCIAL   *
2022-04-07 19:54:43,633 - UOCPRA01 - INFO - ********************************
2022-04-07 19:54:43,633 - UOCPRA01 - INFO - El proceso

In [24]:
# 03.02.-PROCESO USANDO MULTIPROCESO de generación Datos Master

from multiprocessing import Pool

In [25]:
# Lista de web a analizar

# Construcción Lista Webs a analizar
listadoWebs = df['Pag_Referencia'].tolist()


# Para hacer pruebas limitando el alcance listadoWebs[:limite]
url_list=listadoWebs

In [26]:
url_list

['https://www.emagister.com/master-mba-direccion-centros-hidrotermales-cursos-3572081.htm',
 'https://www.emagister.com/master-nutricion-salud-cursos-3693630.htm',
 'https://www.emagister.com/master-biotecnologia-salud-madrid-cursos-3170196.htm',
 'https://www.emagister.com/master-universitario-nutricion-salud-cursos-3006594.htm',
 'https://www.emagister.com/gestion-sanitaria-para-profesionales-salud-cursos-2801676.htm',
 'https://www.emagister.com/tecnico-salud-mental-10-creditos-ects-cursos-3548099.htm',
 'https://www.emagister.com/master-salud-mental-anciano-master-enfermedad-del-alzheimer-cursos-3640936.htm',
 'https://www.emagister.com/master-salud-mental-cursos-3466528.htm',
 'https://www.emagister.com/master-nutricion-salud-distancia-cursos-3693631.htm',
 'https://www.emagister.com/master-psicologia-clinica-salud-cursos-3548010.htm',
 'https://www.emagister.com/master-trabajo-social-servicios-salud-master-salud-mental-cursos-3658125.htm',
 'https://www.emagister.com/master-inter

In [27]:
# https://medium.com/@kunal.rustagi/boost-your-web-crawler-using-multiple-processes-in-python-3cc3ff519226

start = time.time()

logger.info('********************************')
logger.info('*   INICIO MULTIPROCESO        *')
logger.info('********************************')

# Numero procesos simultaneos
num_proc = 10
logger.info('Procesos simultaneos: %s',str(num_proc))


p = Pool(num_proc)
resultados_mul = p.map(WebSecundaria, url_list)
p.terminate()
p.join()



logger.info('********************************')
logger.info('*     FIN MULTIPROCESO         *')
logger.info('********************************')

end = time.time()
duracion = (end-start)
logger.info('La secuencia Multiproceso duro %s segundos', duracion)

2022-04-07 19:54:43,682 - UOCPRA01 - INFO - ********************************
2022-04-07 19:54:43,682 - UOCPRA01 - INFO - *   INICIO MULTIPROCESO        *
2022-04-07 19:54:43,683 - UOCPRA01 - INFO - ********************************
2022-04-07 19:54:43,684 - UOCPRA01 - INFO - Procesos simultaneos: 10
2022-04-07 19:54:46,086 - UOCPRA01 - INFO - Procesada  Máster en estrategia de comunicación digital 
2022-04-07 19:54:46,216 - UOCPRA01 - INFO - Procesada  Máster MBA en dirección de centros hidrotermales 
2022-04-07 19:54:46,223 - UOCPRA01 - INFO - Procesada  Máster de formación permanente en marketing online y e-Commerce 
2022-04-07 19:54:46,236 - UOCPRA01 - INFO - Procesada  Máster en detección y diagnóstico de patologías del habla y el lenguaje + Máster en trastornos del lenguaje, el habla y la comunicación 
2022-04-07 19:54:46,368 - UOCPRA01 - INFO - Procesada  Diplomado en calidad total 
2022-04-07 19:54:46,373 - UOCPRA01 - INFO - Procesada  Máster en análisis clínicos en laboratorio p

2022-04-07 19:55:04,163 - UOCPRA01 - INFO - Procesada  MÁSTER EN SALUD MENTAL 
2022-04-07 19:55:04,513 - UOCPRA01 - INFO - Procesada  Master En Gestión Y Dirección De Clínicas Veterinarias + Prevención De Riesgos Laborales En Veterinaria 
2022-04-07 19:55:04,620 - UOCPRA01 - INFO - Procesada  Máster en Nutrición y Salud 
2022-04-07 19:55:05,493 - UOCPRA01 - INFO - Procesada 
                     Programa Ejecutivo en Digital Customer Experience             
2022-04-07 19:55:05,572 - UOCPRA01 - INFO - Procesada  Máster en rehabilitación deportiva 
2022-04-07 19:55:05,836 - UOCPRA01 - INFO - Procesada  Máster en Comercio Internacional 
2022-04-07 19:55:06,191 - UOCPRA01 - INFO - Procesada  Máster en estrategia de comunicación digital 
2022-04-07 19:55:06,337 - UOCPRA01 - INFO - Procesada  Máster en Nutrición y Salud - A distancia 
2022-04-07 19:55:07,055 - UOCPRA01 - INFO - Procesada  Máster universitario en investigación en psicología 
2022-04-07 19:55:07,258 - UOCPRA01 - INFO - Procesa

2022-04-07 19:55:26,843 - UOCPRA01 - INFO - Procesada  Máster en dirección de centros médicos, clínicas y hospitales más máster en gerencia de la salud 
2022-04-07 19:55:27,121 - UOCPRA01 - INFO - Procesada  Máster En Gestión Sanitaria - con Prácticas Garantizadas 
2022-04-07 19:55:27,261 - UOCPRA01 - INFO - Procesada  Posgrado Experto en Nuevas Tecnologías para Directivos Pymes + Máster en Coaching Tecnológico 
2022-04-07 19:55:27,374 - UOCPRA01 - INFO - Procesada  Máster en psicología infantil y adolescente más máster en coaching y en inteligencia emocional infantil y juvenil 
2022-04-07 19:55:28,112 - UOCPRA01 - INFO - Procesada  Máster Universitario en Marketing Digital 
2022-04-07 19:55:28,371 - UOCPRA01 - INFO - Procesada  Posgrado de Dirección y Gestión de la Calidad 
2022-04-07 19:55:28,463 - UOCPRA01 - INFO - Procesada  MÁSTER EN PROJECT MANAGER PROFESIONAL 
2022-04-07 19:55:28,537 - UOCPRA01 - INFO - Procesada  Máster online de experto en nutrición (con certificado) 
2022-04-

2022-04-07 19:55:48,542 - UOCPRA01 - INFO - Procesada  Máster Universitario en Dirección y Gestión de Unidades de Enfermería 
2022-04-07 19:55:48,556 - UOCPRA01 - INFO - Procesada  Máster en dirección estratégica y gestión de proyectos 
2022-04-07 19:55:54,340 - UOCPRA01 - INFO - Procesada  POSTGRADO EXPERTO EN AUTÓMATAS PROGRAMABLES 
2022-04-07 19:55:54,445 - UOCPRA01 - INFO - Procesada  Posgrado en project management 
2022-04-07 19:55:54,455 - UOCPRA01 - INFO - Procesada  MÁSTER EN DIRECCIÓN Y GESTIÓN SANITARIA 
2022-04-07 19:55:54,496 - UOCPRA01 - INFO - Procesada  Master en Nutrición Pediátrica 
2022-04-07 19:55:54,639 - UOCPRA01 - INFO - Procesada  MBA - MASTER EN ADMINISTRACIÓN DE EMPRESAS 
2022-04-07 19:55:54,680 - UOCPRA01 - INFO - Procesada  Global Master en Recursos Humanos y Desarrollo de Talento 
2022-04-07 19:55:54,729 - UOCPRA01 - INFO - Procesada  Master en Gestión de Procesos Administrativos y Financieros en el Comercio Internacional + Titulación Universitaria 
2022-04-

2022-04-07 19:56:28,868 - UOCPRA01 - INFO - Procesada  Máster en autómatas programables 
2022-04-07 19:56:28,887 - UOCPRA01 - INFO - Procesada  Máster En Creación De Empresas + Máster En Dirección Y Creación De Start - UPS - con Prácticas Garantizadas 
2022-04-07 19:56:31,569 - UOCPRA01 - INFO - Procesada  Máster Experto en Logopedia + Máster en Atención Temprana 
2022-04-07 19:56:32,098 - UOCPRA01 - INFO - Procesada  Máster en atención de enfermería en urgencias y emergencias 
2022-04-07 19:56:32,485 - UOCPRA01 - INFO - Procesada  Salud Pública y Comunitaria para Médicos y Personal de Enfermería ONLINE 
2022-04-07 19:56:33,214 - UOCPRA01 - INFO - Procesada  -MBA - Máster en Administración y Dirección de Empresas- 
2022-04-07 19:56:33,631 - UOCPRA01 - INFO - Procesada  Máster en Técnicas Auxiliares de Odontología 
2022-04-07 19:56:35,993 - UOCPRA01 - INFO - Procesada  Máster en Dirección y Gestión de Proyectos - Project Manager 
2022-04-07 19:56:36,100 - UOCPRA01 - INFO - Procesada  Má

...

2022-04-07 20:09:39,022 - UOCPRA01 - INFO - Procesada 
                     Curso de Francés. Utilisateur Indépendant B2             
2022-04-07 20:09:39,137 - UOCPRA01 - INFO - Procesada  Máster de formación permanente en logística y comercio internacional 
2022-04-07 20:09:39,388 - UOCPRA01 - INFO - Procesada  Máster en personal trainer y coach deportivo 
2022-04-07 20:09:39,559 - UOCPRA01 - INFO - Procesada  Clases particulares de italiano 
2022-04-07 20:09:39,571 - UOCPRA01 - INFO - Procesada  Máster en investigación de incendios forestales 
2022-04-07 20:09:39,865 - UOCPRA01 - INFO - Procesada 
                     RECEPCIONISTA + JEFE DE RECEPCIÓN             
2022-04-07 20:09:40,229 - UOCPRA01 - INFO - Procesada  POSTGRADO AUXILIAR DE TOPOGRAFÍA 
2022-04-07 20:09:40,512 - UOCPRA01 - INFO - Procesada  Producción y venta de servicios turísticos en agencia de viajes(CON PRÁCTICAS) 
2022-04-07 20:09:40,648 - UOCPRA01 - INFO - Procesada  Máster en personal trainer y coaching deportiv

2022-04-07 20:09:59,213 - UOCPRA01 - INFO - Procesada  Produccion y venta de servicios turisticos en agencia de viajes (CURSO ONLINE CON PRÁCTICAS PRESENCIALES) 
2022-04-07 20:09:59,821 - UOCPRA01 - INFO - Procesada  CURSO DE PILATES TERAPÉUTICO + MÁSTER EN NUTRICIÓN DEPORTIVA Y COACH NUTRICIONAL- Con prácticas garantizadas 
2022-04-07 20:10:00,122 - UOCPRA01 - INFO - Procesada  Máster de formación permanente en logística y transporte 
2022-04-07 20:10:00,177 - UOCPRA01 - INFO - Procesada 
                     CURSO DE INGLÉS: A2             
2022-04-07 20:10:00,258 - UOCPRA01 - INFO - Procesada  Máster oficial en ciencia política y gestión pública (URJC) 
2022-04-07 20:10:00,645 - UOCPRA01 - INFO - Procesada  Máster en psicopatología criminal 
2022-04-07 20:10:00,780 - UOCPRA01 - INFO - Procesada  Clases de Conversación Online de Preparación del Nivel C1 de Italiano 
2022-04-07 20:10:01,091 - UOCPRA01 - INFO - Procesada  MÁSTER EXPERTO EN TURISMO LOCAL 
2022-04-07 20:10:01,540 - UOCPR

2022-04-07 20:10:20,384 - UOCPRA01 - INFO - Procesada  MÁSTER EN ELABORACIÓN DE VINOS 
2022-04-07 20:10:20,429 - UOCPRA01 - INFO - Procesada  Master en Actividades de Natación 
2022-04-07 20:10:20,580 - UOCPRA01 - INFO - Procesada  Ingles Básico 
2022-04-07 20:10:20,704 - UOCPRA01 - INFO - Procesada  Máster Oficial - Tradumática: Tecnologías de la Traducción 
2022-04-07 20:10:20,958 - UOCPRA01 - INFO - Procesada  Procesos y estrategias de aprendizaje de ELE 
2022-04-07 20:10:21,225 - UOCPRA01 - INFO - Procesada  Máster en coaching y psicología deportiva 
2022-04-07 20:10:21,648 - UOCPRA01 - INFO - Procesada  Máster en Tráfico, Movilidad y Seguridad Vial 
2022-04-07 20:10:21,921 - UOCPRA01 - INFO - Procesada  Master Oficial Universitario en Policía Científica e Inteligencia Criminal + 60 Créditos ECTS 
2022-04-07 20:10:22,715 - UOCPRA01 - INFO - Procesada  Máster en administración pública 
2022-04-07 20:10:22,870 - UOCPRA01 - INFO - Procesada  Máster en Artes Culinarias y Dirección de C

2022-04-07 20:10:41,183 - UOCPRA01 - INFO - Procesada  Cocina en Línea Fría - ONLINE 
2022-04-07 20:10:41,564 - UOCPRA01 - INFO - Procesada  Master Experto En Transporte Sanitario + Master En Primeros Auxilios 
2022-04-07 20:10:41,794 - UOCPRA01 - INFO - Procesada  Máster en Enseñanza del Español como Lengua Extranjera 
2022-04-07 20:10:42,022 - UOCPRA01 - INFO - Procesada 
                     RECEPCIONISTA + JEFE DE RECEPCIÓN             
2022-04-07 20:10:42,203 - UOCPRA01 - INFO - Procesada  Máster universitario en gestión de contratos y programas en el sector público, con especial aplicación al ámbito de la defensa 
2022-04-07 20:10:42,628 - UOCPRA01 - INFO - Procesada  Máster en mediación social con menores - CON PRÁCTICAS GARANTIZADAS 
2022-04-07 20:10:42,767 - UOCPRA01 - INFO - Procesada  Monitor y Guía De Bicicleta y Senderismo 
2022-04-07 20:10:42,827 - UOCPRA01 - INFO - Procesada  Máster Oficial - Traducción Audiovisual 
2022-04-07 20:10:42,843 - UOCPRA01 - INFO - Procesada  

2022-04-07 20:11:03,967 - UOCPRA01 - INFO - Procesada  MÁSTER EN ESTRATEGIA, INNOVACIÓN Y CONTROL DE GESTIÓN. 
2022-04-07 20:11:04,269 - UOCPRA01 - INFO - Procesada  Máster en dirección hotelera y turismo 
2022-04-07 20:11:04,433 - UOCPRA01 - INFO - Procesada  Máster Internacional Online en Paz Seguridad y Defensa 
2022-04-07 20:11:04,999 - UOCPRA01 - INFO - Procesada  Lengua de signos española 
2022-04-07 20:11:05,734 - UOCPRA01 - INFO - Procesada  Master en Sistemas Integrados de Administración Financiera - SIAF + Titulación Universitaria 
2022-04-07 20:11:06,219 - UOCPRA01 - INFO - Procesada  Máster Universitario en Enseñanza de Inglés como Lengua Extranjera 
2022-04-07 20:11:06,542 - UOCPRA01 - INFO - Procesada  Máster Universitario en Gestión Hotelera Online 
2022-04-07 20:11:06,815 - UOCPRA01 - INFO - Procesada  PROFESOR DE ESPAÑOL PARA EXTRANJEROS -ELE -' 
2022-04-07 20:11:07,132 - UOCPRA01 - INFO - Procesada  Máster en pilotaje de drones 
2022-04-07 20:11:07,226 - UOCPRA01 - IN

In [28]:
# 03.03.- Creación Dataframe resultado

DfResult = pd.DataFrame(resultados_mul, columns = ['MASTER',
                                                   'ENTIDAD',
                                                   'PRECIO',
                                                   'TIPOLOGIA',
                                                   'METODOLOGIA',
                                                   'DURACION',
                                                   'BOLSAEMP',
                                                   'TELEFONO',
                                                   'DESCR',
                                                   'PROGRAMA',
                                                   'URLIMAGEN',
                                                   'NOMBREIMAGEN',
                                                   'WEB'])

In [29]:
DfResult

,MASTER,ENTIDAD,PRECIO,TIPOLOGIA,METODOLOGIA,DURACION,BOLSAEMP,TELEFONO,DESCR,PROGRAMA,URLIMAGEN,NOMBREIMAGEN,WEB
0,Máster MBA en dirección de centros hidrotermales,SELECT BUSINESS SCHOOL,1940.0,Master,Semipresencial,Madrid,Sí,916358357,Plantea tus dudas y otros usuarios podran resp...,"[Contenido formativo, PARTE 1. GESTIÓN LABORAL...",https://www.emagister.com/assets/es/logos/cent...,['l.png'],https://www.emagister.com/master-mba-direccion...
1,Máster en Nutrición y Salud,ISEB - Instituto Superior Europeo de Barcelona,1250.0,Master,Online,12 Meses,Sin Datos,932824831,Plantea tus dudas y otros usuarios podran resp...,[],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-nutricion-sal...
2,Máster en Biotecnología de la Salud - Madrid,CESIF Formación,Precio a consultar,Master,Madrid,1840h,Sin Datos,912669065,Plantea tus dudas y otros usuarios podran resp...,[SECTOR INDUSTRIAL EN BIOTECNOLOGIA DE LA SALU...,https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-biotecnologia...
3,Máster Universitario en Nutrición y Salud,Universidad Internacional de Valencia (VIU),Precio a consultar,Master oficial,Online,Sin Datos,Sin Datos,962642879,El principal objetivo del Master Oficial Onlin...,[],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-universitario...
4,Máster Profesional en Gestión Sanitaria,Cef - Centro de Estudios Financieros,5400.0,Master oficial,Online,Sin Datos,Sin Datos,912669065,Si trabajas en el sector sanitario y quieres a...,[Programa \n\n],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/gestion-sanitaria-pa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3088,MÁSTER EN SEGURIDAD EN PARQUES INFANTILES,ESNECA BUSINESS SCHOOL,380.0,Master,A distancia,600h,Sin Datos,973975855,Plantea tus dudas y otros usuarios podran resp...,[],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-seguridad-par...
3089,Master Oficial Universitario en Policía Cientí...,UB - Universitat de Barcelona,3495.0,Master,Barcelona,Sin Datos,Sin Datos,932933037,ENALDE BUSINESS SCHOOL,[],https://www.emagister.com/assets/es/logos/cent...,['l.png'],https://www.emagister.com/master-oficial-polic...
3090,Máster Universitario en Seguridad y Defensa,A+ CAPEC,25700.0,Master,Sydney (Australia),Sin Datos,Sin Datos,912669065,Universidad Nebrija,"[Asignaturas Troncales, Asignaturas Troncales,...",https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-of-internatio...
3091,Máster en derecho administrativo,Acath Formació,2001.0,Postgrado,Santa coloma de farners,Sin Datos,Sin Datos,932933037,ESCUELA DE POSTGRADO DE CIENCIAS DEL DERECHO,[],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/postgrado-adiestrami...


In [30]:
DfResult.columns

Index(['MASTER', 'ENTIDAD', 'PRECIO', 'TIPOLOGIA', 'METODOLOGIA', 'DURACION',
       'BOLSAEMP', 'TELEFONO', 'DESCR', 'PROGRAMA', 'URLIMAGEN',
       'NOMBREIMAGEN', 'WEB'],
      dtype='object')

In [31]:
# 03.04.01.- Generación Archivo parcial de resultados (.xlsx)

# Generación Nombres Archivos a generar
from datetime import datetime

nombre=datetime.today().strftime('%d-%m-%y')

FILE_02 = "InfoMaster-"+nombre+".xlsx"

# Directorio donde se generara salida
path='RESULTADOS'
file1=GenDirecFile(path, FILE_02)

# Escritura
DfResult.to_excel(file1)
logger.info('Generado Archivo %s', FILE_02)

2022-04-07 20:11:40,209 - UOCPRA01 - INFO - Generado Archivo InfoMaster-07-04-22.xlsx


In [32]:
# 03.04.02.- Generación Archivo parcial de resultados (CSV)

# Generación Nombres Archivos a generar
from datetime import datetime

nombre=datetime.today().strftime('%d-%m-%y')

FILE_03 = "InfoMaster-"+nombre+".csv"

# Directorio donde se generara salida
path='RESULTADOS'
filecsv=GenDirecFile(path, FILE_03)

DfResult.to_csv(filecsv, index = False)
logger.info('Generado Archivo %s', FILE_03)

2022-04-07 20:11:40,272 - UOCPRA01 - INFO - Generado Archivo InfoMaster-07-04-22.csv


In [22]:
# Extracción Imagenes de las WEBs

# Construccion diccionario imagenes de las entidades

# Filtrado Entidades con Logo
#DfResultLOGO=DfResult[DfResult['URLIMAGEN']!='']

# Lista entidades unicas
listaOrganizaciones = DfResult['ENTIDAD'].unique().tolist()

# Construccion diccionario imagenes de las entidades
DiccioImagenes = {}
# Para cada entidad
for s in listaOrganizaciones:
    # Hacer entrada por entidad con lista de imagenes de la entidad
    DiccioImagenes[s] = DfResult[DfResult['ENTIDAD'] == s]['URLIMAGEN'].unique().tolist()
    
logger.info('Generado Diccionario de tamaño %s', len(DiccioImagenes))   



2022-04-02 21:10:11,963 - UOCPRA01 - INFO - Generado Diccionario de tamaño 195


In [23]:
# Prueba

DiccioImagenes['Instituto Español de Formación Social']

['https://www.emagister.com/assets/es/logos/centro/id/192475/size/l.png']

In [24]:
# Directorio Imagenes
Path = 'IMAGENES'

logger.info('Inicio Descarga Imagenes')

# Itera en el diccionario Previo
for k, v in DiccioImagenes.items():
    # Descarga Archivo en Path
    # v --> Lista de Links de imagenes
    # k --> Institucion
    # Archivo resultante con nombre
    logger.info('Descargando Logo de %s', k)
    # Pasar lista a String
    v = v[0]
    logger.info('Link %s', v)
    # - Eliminamos caracteres especiales
    k=re.sub(r"[^a-zA-Z0-9]"," ",k)
    # - Elimina espacios blanco principio y final
    k=k.strip()
    DescargaImagenes(Path, v, k)

logger.info('Fin Descarga Imagenes')

2022-04-02 21:10:11,978 - UOCPRA01 - INFO - Inicio Descarga Imagenes
2022-04-02 21:10:11,980 - UOCPRA01 - INFO - Descargando Logo de SELECT BUSINESS SCHOOL
2022-04-02 21:10:11,981 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/192308/size/l.png
2022-04-02 21:10:12,988 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:10:13,210 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/192308/size/l.png HTTP/1.1" 200 13231
2022-04-02 21:10:13,215 - UOCPRA01 - INFO - Descargando Logo de CESIF Formación
2022-04-02 21:10:13,216 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/56155/size/l.jpg
2022-04-02 21:10:15,223 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:10:15,486 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/56155/size/l.jp

2022-04-02 21:10:42,470 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/193808/size/l.jpg HTTP/1.1" 200 6330
2022-04-02 21:10:42,474 - UOCPRA01 - INFO - Descargando Logo de ESCUELA INTERNACIONAL DE FARMACIA PASTEUR
2022-04-02 21:10:42,476 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1140014/size/l.jpg
2022-04-02 21:10:43,479 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:10:43,686 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1140014/size/l.jpg HTTP/1.1" 200 4908
2022-04-02 21:10:43,690 - UOCPRA01 - INFO - Descargando Logo de Editorial Médica Panamericana
2022-04-02 21:10:43,692 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/94945/size/l.jpg
2022-04-02 21:10:44,697 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21

2022-04-02 21:11:10,787 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1721/size/l.jpg HTTP/1.1" 200 19366
2022-04-02 21:11:10,792 - UOCPRA01 - INFO - Descargando Logo de Universidad a Distancia de Madrid - Masters
2022-04-02 21:11:10,793 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/202616/size/l.jpg
2022-04-02 21:11:12,800 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:11:13,056 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/202616/size/l.jpg HTTP/1.1" 200 14787
2022-04-02 21:11:13,058 - UOCPRA01 - INFO - Descargando Logo de VEIGLER BUSINESS SCHOOL
2022-04-02 21:11:13,059 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1153673/size/l.jpg
2022-04-02 21:11:15,065 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:11:

2022-04-02 21:11:43,226 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1168562/size/l.jpg HTTP/1.1" 200 21678
2022-04-02 21:11:43,232 - UOCPRA01 - INFO - Descargando Logo de SEFHOR
2022-04-02 21:11:43,234 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1153674/size/l.jpg
2022-04-02 21:11:44,240 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:11:44,509 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1153674/size/l.jpg HTTP/1.1" 200 30966
2022-04-02 21:11:44,511 - UOCPRA01 - INFO - Descargando Logo de Edil formación
2022-04-02 21:11:44,512 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1123086/size/l.jpg
2022-04-02 21:11:46,518 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:11:46,737 - urllib3.connectionpool - DEBUG -

2022-04-02 21:12:13,637 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/55701/size/l.jpg HTTP/1.1" 200 8541
2022-04-02 21:12:13,642 - UOCPRA01 - INFO - Descargando Logo de Escuela de Postgrado de Veterinaria
2022-04-02 21:12:13,644 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1152427/size/l.jpg
2022-04-02 21:12:14,650 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:12:14,863 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1152427/size/l.jpg HTTP/1.1" 200 6882
2022-04-02 21:12:14,868 - UOCPRA01 - INFO - Descargando Logo de Instituto Europeo de Química, Física y Biología - IEQFB
2022-04-02 21:12:14,869 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1155074/size/l.jpg
2022-04-02 21:12:15,876 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.

...

2022-04-02 21:13:40,405 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:13:40,617 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1160425/size/l.jpg HTTP/1.1" 200 8531
2022-04-02 21:13:40,621 - UOCPRA01 - INFO - Descargando Logo de Spain Business School - Formación
2022-04-02 21:13:40,623 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/139196/size/l.jpg
2022-04-02 21:13:42,630 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:13:42,894 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/139196/size/l.jpg HTTP/1.1" 200 31740
2022-04-02 21:13:42,899 - UOCPRA01 - INFO - Descargando Logo de ESCUELA DE POSTGRADO DE INGENIERIA Y ARQUITECTURA
2022-04-02 21:13:42,901 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1155659/size/l.jpg

2022-04-02 21:14:09,697 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:14:09,962 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1137833/size/l.jpg HTTP/1.1" 200 46066
2022-04-02 21:14:10,013 - UOCPRA01 - INFO - Descargando Logo de Instituto Idem
2022-04-02 21:14:10,014 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/60835/size/l.jpg
2022-04-02 21:14:11,019 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:14:11,235 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/60835/size/l.jpg HTTP/1.1" 200 8209
2022-04-02 21:14:11,238 - UOCPRA01 - INFO - Descargando Logo de SAN ANASTASIO INTERNATIONAL SCHOOL
2022-04-02 21:14:11,239 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1152478/size/l.jpg
2022-04-02 21:14:12,244 - urllib3.c

2022-04-02 21:14:39,099 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1168850/size/l.jpg
2022-04-02 21:14:40,105 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:14:40,321 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1168850/size/l.jpg HTTP/1.1" 200 6620
2022-04-02 21:14:40,326 - UOCPRA01 - INFO - Descargando Logo de THREE POINTS
2022-04-02 21:14:40,328 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1137887/size/l.jpg
2022-04-02 21:14:42,334 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:14:42,551 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1137887/size/l.jpg HTTP/1.1" 200 9297
2022-04-02 21:14:42,556 - UOCPRA01 - INFO - Descargando Logo de CEAM
2022-04-02 21:14:42,558 - UOCPRA01 - INFO - Link https://www.ema

2022-04-02 21:15:07,400 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/59251/size/l.jpg
2022-04-02 21:15:09,407 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:15:09,669 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/59251/size/l.jpg HTTP/1.1" 200 10222
2022-04-02 21:15:09,674 - UOCPRA01 - INFO - Descargando Logo de AITEX Instituto Tecnológico Textil
2022-04-02 21:15:09,675 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/81552/size/l.jpg
2022-04-02 21:15:10,681 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:15:10,895 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/81552/size/l.jpg HTTP/1.1" 200 5796
2022-04-02 21:15:10,900 - UOCPRA01 - INFO - Descargando Logo de UNED
2022-04-02 21:15:10,902 - UOCPRA01 - INFO - Link 

2022-04-02 21:15:39,747 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1154628/size/l.jpg
2022-04-02 21:15:40,753 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:15:40,969 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/1154628/size/l.jpg HTTP/1.1" 200 7376
2022-04-02 21:15:40,974 - UOCPRA01 - INFO - Descargando Logo de UDL - Universitat de Lleida. Màsters Oficials
2022-04-02 21:15:40,975 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/56021/size/l.jpg
2022-04-02 21:15:41,981 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.emagister.com:443
2022-04-02 21:15:42,189 - urllib3.connectionpool - DEBUG - https://www.emagister.com:443 "GET /assets/es/logos/centro/id/56021/size/l.jpg HTTP/1.1" 200 4729
2022-04-02 21:15:42,194 - UOCPRA01 - INFO - Descargando Logo de Net-Learning - Entornos virtuales de apr